In [1]:
# Import modules.
import pandas as pd

import geopandas as gpd

# Geocoding Module.
import pgeocode

# Map plotting module.
import folium
from folium.plugins import MarkerCluster

import json

In [2]:
# Read the data in and set the geocoder to USA.
df = pd.read_excel("/Users/itunuolaitan/Desktop/Portfolio/Code/Data/Folium/us-cities-demographics.xlsx", sheet_name='Sheet1')
nomi = pgeocode.Nominatim('us')

In [3]:
df['Female Population'] = df['Female Population'].astype(int)
df['Male Population'] = df['Male Population'].astype(int)
df['Average Age'] = df['Average Age'].astype(int)
df['Average Household Size'] = df['Average Household Size'].round(1)

In [4]:
df['Female Population'] = df['Female Population'].map('{:,d}'.format)
df['Male Population'] = df['Male Population'].map('{:,d}'.format)
df['Total Population'] = df['Total Population'].map('{:,d}'.format)

In [4]:
df_read = df.copy()
# Create latitude and longitude columns from querying the zip code.
#df_read[['Latitude', 'Longitude']] = nomi.query_postal_code(df['ZIP CODE'].to_list())[['latitude', 'longitude']]
# Drop any rows that don't have latitude and longitude values.
#df_read = df_read.dropna(subset=['Longitude', 'Latitude'])

In [5]:
df_read.head(4)

,name,Male Population,Female Population,Total Population,Average Household Size,Average Age
0,Alabama,2448200,2715106,5163306,2.4,36
1,Alaska,764725,728750,1493475,2.8,32
2,Arizona,11137275,11360435,22497710,2.8,35
3,Arkansas,1400724,1482165,2882889,2.5,32


In [6]:
us_geo = gpd.read_file("/Users/itunuolaitan/Desktop/Portfolio/Code/Folium/us.json")
#extract name and geometry
us_geo = us_geo[['name','geometry']]
us_geo.head(1)

,name,geometry
0,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."


In [7]:
#merge geo state data ad policy data 
data = us_geo.merge(df_read,on="name")
data.head(6)

,name,geometry,Male Population,Female Population,Total Population,Average Household Size,Average Age
0,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2448200,2715106,5163306,2.4,36
1,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",764725,728750,1493475,2.8,32
2,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",11137275,11360435,22497710,2.8,35
3,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",1400724,1482165,2882889,2.5,32
4,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",61055672,62388681,123444353,3.1,36
5,Colorado,"POLYGON ((-107.91973 41.00391, -105.72895 40.9...",7273095,7405250,14678345,2.6,35


In [8]:
# Assigning centroids to map
x_map=data.centroid.x.mean()
y_map=data.centroid.y.mean()
print(x_map,y_map)

-93.89214633096361 39.46734767781407


<ipython-input-8-fa1cee95690c>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x_map=data.centroid.x.mean()
<ipython-input-8-fa1cee95690c>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y_map=data.centroid.y.mean()


In [9]:
# Set the map and the zoom level and set the cluster to add them to the map.
map = folium.Map(location=[y_map, x_map], zoom_start=5)

folium.Choropleth(
    geo_data=us_geo,
    name="chloropleth",
    data=df_read,
    columns=["name", "Total Population"],
    key_on="feature.properties.name",
    fill_color="YlGn",
    nan_fill_color="grey",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Total Population",
).add_to(map)

g = folium.GeoJson(
    data,
    style_function = lambda x: {'fillColor': '#000000', 
                            'color':'#000000', 
                            'fillOpacity': 0.0, 
                            'weight': 0.1},
    highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.0, 
                                'weight': 0.1},
    name='geojson'
    ).add_to(map)


folium.GeoJsonTooltip(
    fields=['name',
            'Total Population',
            'Female Population',
            'Male Population',
            'Average Age',
            'Average Household Size'],
    aliases=['State:',
            'Total Population:',
            'Female Population:',
            'Male Population:',
            'Average Age:',
            'Average Household Size:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
).add_to(g)


folium.LayerControl().add_to(map)

map.add_child(g)
map.keep_in_front(g)


# Save the map.
map.save('All States.html')

In [11]:
df_read.head(60)

,name,Male Population,Female Population,Total Population,Average Household Size,Average Age
0,Alabama,"2,448,200","2,715,106","5,163,306",2.4,36
1,Alaska,"764,725","728,750","1,493,475",2.8,32
2,Arizona,"11,137,275","11,360,435","22,497,710",2.8,35
3,Arkansas,"1,400,724","1,482,165","2,882,889",2.5,32
4,California,"61,055,672","62,388,681","123,444,353",3.1,36
5,Colorado,"7,273,095","7,405,250","14,678,345",2.6,35
6,Connecticut,"2,123,435","2,231,661","4,355,096",2.7,35
7,Delaware,"163,400","196,385","359,785",2.4,36
8,District of Columbia,"1,598,525","1,762,615","3,361,140",2.2,33
9,Florida,"15,461,937","16,626,425","32,088,362",2.8,39
